## Homework [DRAFT]

> Note: sometimes your answer doesn't match one of the options exactly. That's fine. 
Select the option that's closest to your solution.


### Dataset

In this homework, we will use the Car price dataset. Download it from [here](https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv).

Or you can do it with `wget`:

```bash
wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
```

We'll keep working with the `MSRP` variable, and we'll transform it to a classification task. 

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv'
!wget $data

--2024-10-14 19:16:59--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.41M  --.-KB/s    in 0.01s   

2024-10-14 19:16:59 (120 MB/s) - ‘data.csv’ saved [1475504/1475504]



### Features

For the rest of the homework, you'll need to use only these columns:

* `Make`,
* `Model`,
* `Year`,
* `Engine HP`,
* `Engine Cylinders`,
* `Transmission Type`,
* `Vehicle Style`,
* `highway MPG`,
* `city mpg`,
* `MSRP`

### Data preparation

* Select only the features from above and transform their names using the next line:
  ```
  data.columns = data.columns.str.replace(' ', '_').str.lower()
  ```
* Fill in the missing values of the selected features with 0.
* Rename `MSRP` variable to `price`.

In [3]:
import pandas as pd

In [34]:
df = pd.read_csv('/workspaces/MLZoomCamp/03-classification/homework/data.csv')

In [35]:
df.columns

Index(['Make', 'Model', 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Market Category', 'Vehicle Size', 'Vehicle Style',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP'],
      dtype='object')

In [36]:
df_hw = df[['Make', 'Model', 'Year','Engine HP','Engine Cylinders', 'Transmission Type',
            'Vehicle Style','highway MPG', 'city mpg','MSRP']]
df_hw = df_hw.fillna(0)
df_hw = df_hw.rename(columns={'MSRP':'price'})
df_hw.columns = df_hw.columns.str.replace(' ', '_').str.lower()

### Question 1

What is the most frequent observation (mode) for the column `transmission_type`?

- `AUTOMATIC`
- MANUAL
- AUTOMATED_MANUAL
- DIRECT_DRIVE

In [37]:
df_hw.transmission_type.value_counts()

transmission_type
AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: count, dtype: int64

### Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset. 
In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

What are the two features that have the biggest correlation in this dataset?

- engine_hp and year
- engine_hp and engine_cylinders
- highway_mpg and engine_cylinders
- `highway_mpg and city_mpg`

In [38]:
df_hw[['year','engine_cylinders']].corrwith(df_hw['engine_hp'])

year                0.338714
engine_cylinders    0.774851
dtype: float64

In [39]:
df_hw[['city_mpg','engine_cylinders']].corrwith(df_hw['highway_mpg'])

city_mpg            0.886829
engine_cylinders   -0.614541
dtype: float64

### Make `price` binary

* Now we need to turn the `price` variable from numeric into a binary format.
* Let's create a variable `above_average` which is `1` if the `price` is above its mean value and `0` otherwise.

In [45]:
df_hw['above_average'] = (df_hw['price'] > df_hw['price'].mean()).astype(int)

### Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
* Make sure that the target value (`above_average`) is not in your dataframe.

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
df_full_train, df_test = train_test_split(df_hw, test_size=0.2, random_state=42)

In [54]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [55]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_full_train = df_full_train.reset_index(drop=True)
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values
del df_train['above_average']
del df_val['above_average']
del df_test['above_average']


### Question 3

* Calculate the mutual information score between `above_average` and other categorical variables in our dataset. 
  Use the training set only.
* Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the lowest mutual information score?
  
- make
- model
- transmission_type
- `vehicle_style`

In [56]:
from sklearn.metrics import mutual_info_score

In [57]:
mutual_info_score(df_full_train.above_average, df_full_train.make)

0.2387236479073192

In [58]:
mutual_info_score(df_full_train.above_average, df_full_train.model)

0.46099440756035703

In [59]:
mutual_info_score(df_full_train.above_average, df_full_train.transmission_type)

0.020883575914971142

In [60]:
mutual_info_score(df_full_train.above_average, df_full_train.vehicle_style)

0.08339022741593435

### Question 4

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.60
- 0.72
- 0.84
- 0.95

In [73]:
from sklearn.feature_extraction import DictVectorizer

In [67]:
df_hw.dtypes

make                  object
model                 object
year                   int64
engine_hp            float64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
price                  int64
above_average          int64
dtype: object

In [70]:
categorical = list(df_hw.columns[df_hw.dtypes == 'object'])
numerical = ['year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg']

In [71]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
train_dicts[1]

{'make': 'Nissan',
 'model': 'Rogue',
 'transmission_type': 'AUTOMATIC',
 'vehicle_style': '4dr SUV',
 'year': 2016,
 'engine_hp': 170.0,
 'engine_cylinders': 4.0,
 'highway_mpg': 33,
 'city_mpg': 26}

In [74]:
dv = DictVectorizer(sparse=False)

In [75]:
X_train = dv.fit_transform(train_dicts)

In [78]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [79]:
from sklearn.linear_model import LogisticRegression

In [80]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [87]:
y_pred = model.predict_proba(X_val)[:,1]
price_decision = (y_pred >= 0.5)
price_decision.astype(int)

array([0, 0, 1, ..., 0, 0, 0])

In [89]:
(price_decision == y_val).mean().round(2)

0.92

### Question 5 

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 

Which of following feature has the smallest difference?

- year
- engine_hp
- transmission_type
- `city_mpg`

> **Note**: the difference doesn't have to be positive

In [90]:
data_feature_less_importance = ['year','engine_hp','transmission_type','city_mpg']

In [93]:
df_train_aux = df_train.drop(['price'],axis=1)
df_val_aux = df_val.drop(['price'],axis=1)

In [97]:
for feature in data_feature_less_importance:
    df_small_train = df_train_aux.drop([feature],axis=1)
    df_small_val = df_val_aux.drop([feature],axis=1)
    train_small_dicts = df_small_train.to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_small_train = dv.fit_transform(train_small_dicts)
    val_small_dicts = df_small_val.to_dict(orient='records')
    X_small_val = dv.transform(val_dicts)
    model_small = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model_small.fit(X_small_train, y_train)
    y_small_pred = model_small.predict(X_small_val)
    print(f"Sin feature {feature} score = {(y_small_pred == y_val).mean()}\
           diferencia con el modelo con todas las variables = \
          {(price_decision.astype(int) == y_val).mean().round(2)-(y_small_pred == y_val).mean()}")

Sin feature year score = 0.9429290809903483           diferencia con el modelo con todas las variables =           -0.022929080990348294
Sin feature engine_hp score = 0.9362148552245069           diferencia con el modelo con todas las variables =           -0.01621485522450683
Sin feature transmission_type score = 0.9395719681074276           diferencia con el modelo con todas las variables =           -0.019571968107427562
Sin feature city_mpg score = 0.9341166596726815           diferencia con el modelo con todas las variables =           -0.014116659672681497


### Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn.
* We'll need to use the original column `price`. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model on the training data with a solver `'sag'`. Set the seed to `42`.
* This model also has a parameter `alpha`. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`.
* Round your RMSE scores to 3 decimal digits.

Which of these alphas leads to the best RMSE on the validation set?

- 0
- 0.01
- 0.1
- 1
- `10`

> **Note**: If there are multiple options, select the smallest `alpha`.

In [102]:
import numpy as np

In [108]:
y_train_lr = df_train['price']
y_val_lr = df_val['price']
df_train_lr = df_train.drop(['price'],axis=1)
df_val_lr = df_val.drop(['price'],axis=1)
y_train_lr_log = np.log1p(y_train_lr)
y_val_lr_log = np.log1p(y_val_lr)

In [109]:
train_dicts_lr = df_train_lr.to_dict(orient='records')
dv_lr = DictVectorizer(sparse=False)
X_train_lr = dv.fit_transform(train_dicts_lr)
val_dicts_lr = df_val_lr.to_dict(orient='records')
X_val_lr = dv.transform(val_dicts_lr)


In [110]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [113]:
from sklearn.linear_model import Ridge
values_alpha = [0, 0.01, 0.1, 1, 10]
for alpha in values_alpha:
    model_lr = Ridge(solver='sag',random_state=42,alpha=alpha)
    model_lr.fit(X_train_lr,y_train_lr_log)
    y_pred_lr = model_lr.predict(X_val_lr)
    score = rmse(y_val_lr_log,y_pred_lr)
    print (f'The score for alpha = {alpha} = {score}')


/opt/conda/envs/ml-zoomcamp/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


The score for alpha = 0 = 0.4746151925034722


/opt/conda/envs/ml-zoomcamp/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


The score for alpha = 0.01 = 0.474615413147775


/opt/conda/envs/ml-zoomcamp/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


The score for alpha = 0.1 = 0.4746173989218704


/opt/conda/envs/ml-zoomcamp/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


The score for alpha = 1 = 0.4746372512694775
The score for alpha = 10 = 0.47483523305585146


/opt/conda/envs/ml-zoomcamp/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.4746173989218704